# Advanced plot tools for large datasets
#### Author: Gabriel Luan Souza de Oliveira

#### Last update: 15/02/2022 11:29:13

In [ ]:
from datetime import datetime
today = datetime.today()
print("Today's date:", today.strftime("%d/%m/%Y %H:%M:%S"))

### **For local installation**:
A useful commmand to avoid any import problems (like circular imports) with holoviews, bokh and datashader, use this command on your terminal:
`pip install holoviews[all]`

In [ ]:
# General python imports
import numpy as np
#import pandas as pd
#import warnings

# Update this option setting as you prefer
#pd.set_option('display.max_rows', 5)

import healpy as hp

# Astropy
from astropy.table import Table
#from astropy import units as u
#from astropy.coordinates import SkyCoord
#from astropy.units.quantity import Quantity

# LSST imports
#from lsst.rsp import get_tap_service

# Bokeh and Holoviews for visualization
import bokeh
from bokeh.io import output_notebook, show
#from bokeh.models import ColumnDataSource, Range1d, HoverTool
#from bokeh.models import CDSView, GroupFilter
#from bokeh.plotting import figure, gridplot
#from bokeh.transform import factor_cmap

import holoviews as hv
#from holoviews import streams, opts
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize
#from holoviews.plotting.util import process_cmap

#import datashader as dsh

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

# ***Loading catalog***

In [ ]:
nside=4096
steradian_in_degrees = (180./np.pi)**2
sky_area = 4*np.pi*steradian_in_degrees   #Area of an sphere measured in degrees^2. Read more on https://www.mathsisfun.com/geometry/steradian.html
npixels = hp.nside2npix(nside)            #Númeto de pixeis em um Healpix pixelization scheme => 12.0*(nside)**2.0
area_of_each_pixel = sky_area/(npixels)   #In degrees^2.
pix_area_arcmin = 3600*area_of_each_pixel #Pixel area in (minutes of degree)².

In [ ]:
catalog = Table.read('catalog_table.csv', format='ascii.csv')
np.random.seed(1024)
#p_0 = 0.99
p_0 = 0
random_indexs = np.random.choice(2, size=len(catalog), p=[p_0,1-p_0]).astype('bool')
catalog_reduced = catalog[random_indexs]
del catalog

In [ ]:
print(catalog_reduced)

## Point plot

In [ ]:
#hv.help(hv.Points)
hv.output(size=150)
data_points = zip(catalog_reduced['ra'], catalog_reduced['dec'],
                  catalog_reduced['z_best'],catalog_reduced['err_z'])

pt_1 = hv.Points(data_points, vdims = ['Redshift', 'err_z'])
options_dict = {'title': 'Objects, z and err_z: Point element',
                'toolbar':'above',
                'cmap': 'viridis',
                'color': 'Redshift',
                'marker': 'o',
                'size':0.5,
                'xlabel':'R.A.(degrees)',
                'ylabel': 'DEC. (degrees)',
                'bgcolor': '#f7f7f7',
                'colorbar': True,
                'clim': (0,2),
                'size': hv.dim('err_z'),
                'clabel': 'Redshift'
               }

options_dict_2 = {'title': 'Objects count',
                'toolbar':'above',
                'xlabel':'R.A.(degrees)',
                'ylabel': 'DEC. (degrees)',
                'bgcolor': '#f7f7f7',
                'colorbar': True,
               }

#pt_1.opts(**options_dict)
#pt_1.hist(bins = 'fd')

In [ ]:
rasterize(pt_1).opts(cmap="hot_r", cnorm="eq_hist").relabel().opts(**options_dict_2).hist()

## Scatter plot

In [ ]:
hv.output(size=150)
sc_1 = hv.Scatter((catalog_reduced['ra'], catalog_reduced['dec']), 'R.A.(degrees)','DEC. (degrees)')
#sc_1.opts(color='r', marker='o', size = 0.5)

In [ ]:
rasterize(sc_1[34:36,-6:-4]).opts(cmap="hot_r", cnorm="eq_hist").relabel().opts(**options_dict_2).hist()

In [ ]:
#datashade(pt_1.opts(**options_dict), cmap = 'viridis').hist()

## Object density plot

In [ ]:
density_table = Table()
density_table['pixels'] = np.unique(catalog_reduced['hpix_4096'].astype('int'))

count = np.bincount(catalog_reduced['hpix_4096'].astype('int'))
n_obj = np.array([count[each] for each in density_table['pixels']])


ra,dec = hp.pix2ang(nside=4096, ipix=density_table['pixels'], lonlat = True, nest = True)
ra[ra>180] -= 360

density_table['ra'] = ra
density_table['dec'] = dec
density_table['n_obj'] = n_obj

#print(density_table)
#print(pix_area_arcmin)

In [ ]:
hv.output(size=150)
data_points = zip(density_table['ra'],density_table['dec'],density_table['n_obj']/pix_area_arcmin)
colormaps = ['YlOrRd', 'hot_r']
pt_1 = hv.Points(data_points, vdims = ['N° de objetos'])
options_dict = {'title': 'N° de objetos por arcmin²',
                'toolbar':'above',
                'cmap': colormaps[1],
                'color': 'N° de objetos',
                'marker': 'o',
                'size':0.5,
                'xlabel':'R.A.(degrees)',
                'ylabel': 'DEC. (degrees)',
                'bgcolor': '#f7f7f7',
                'logz': False,
                'colorbar': True,
                'clabel': r'N° de objetos / arcmin²'
               }

#pt_1.opts(**options_dict)
#pt_1.hist(bins = 'fd')

#frequencies, edges = np.histogram(density_table['n_obj']/pix_area_arcmin, bins = 'fd')
#hv.Histogram((edges,frequencies))

In [ ]:
rasterize(pt_1).opts(cmap="hot_r", cnorm="eq_hist", colorbar = True).relabel().opts(**options_dict_2).hist()

In [ ]:
datashade(pt_1.opts(**options_dict), cmap = 'viridis')